In [1]:
import numpy as np
import cv2
from PIL import Image

from Algorithm import key_expansion, ciphering_block, key_update, deciphering_block, key_update_hk, Arnold, s
from Image import blockshaped, From_blocks_to_arr_image
from Metrics import Entr, Hist, EQ

In [2]:
# шифрование картинки

k1 = np.random.randint(0,256,16)
k2 = np.random.randint(0,256,16)
k3 = np.random.randint(0,256,16)
k4 = np.random.randint(0,256,16)

img_path = 'lena_gray.jpg'
img = cv2.imread(img_path, 0)

imgb = blockshaped(img, 16, 16)
Nb = len(imgb)
imgb_c = [0] * Nb
imgb_d = [0] * Nb

key_B = key_expansion(k1, k2, k3, k4)

for i in range(Nb):
    B = imgb[i]
    B_C = ciphering_block(B, key_B, 8)
    imgb_c[i] = B_C
    key_B = key_update(key_B, B_C, B, k1, k2, k3, k4, 8)

In [3]:
k = int(len(img[0]) / 16)

arr2 = From_blocks_to_arr_image(imgb_c,Nb,k)

im_1 = Image.fromarray(arr2, 'L')
im_1.save("encrypted.jpg")

In [4]:
# дешифрование зашифрованной картинки
key_B = key_expansion(k1, k2, k3, k4)

for i in range(Nb):
    B_C = imgb_c[i]
    B_D = deciphering_block(B_C, key_B, 8)
    imgb_d[i] = B_D
    key_B = key_update(key_B, B_C, B_D, k1, k2, k3, k4, 8)

In [5]:
arr2 = From_blocks_to_arr_image(imgb_d,Nb,k)

im_1 = Image.fromarray(arr2, 'L')
im_1.save("decrypted.jpg")

Plain-text атака при постоянном ключе

In [6]:
key_B_0_hk = deciphering_block(imgb_c[0], imgb_d[0], 8)
print((key_B_0_hk==key_expansion(k1, k2, k3, k4)).all())

True


C:\Users\ynov\OneDrive\Рабочий стол\Securing_DICOM_Images\Algorithm.py:19: RuntimeWarning: overflow encountered in ubyte_scalars
  p_i = (c_i - k_i) % (2**n)


In [7]:
def Arnold_hk(key_B):
    key_B_new = Arnold(key_B)
    key_B_new_prev = key_B
    while (key_B_new != key_B).any():
        key_B_new_prev = key_B_new
        key_B_new = Arnold(key_B_new)
    return key_B_new_prev

In [8]:
key_B_1_hk = deciphering_block(imgb_c[1], imgb_d[1], 8)

s_1 = (Arnold_hk(key_B_1_hk) - key_B_0_hk - np.outer(imgb_c[0].sum(0),imgb_d[0].sum(1)))%(2**8)

In [9]:
print((s_1==s(k1, k2, k3, k4)%(2**8)).all()) #Матрица s и полученная нами при анализе s_1 совпадают

True


In [10]:
# дешифрование используя только матрицы, полученные при анализе шифрованой и дешифрованой картинок
key_B_hk = key_B_0_hk

imgb_d_hk = [0] * Nb

for i in range(Nb):
    B_C = imgb_c[i]
    B_D = deciphering_block(B_C, key_B_hk, 8)
    imgb_d_hk[i] = B_D
    key_B_hk = key_update_hk(key_B_hk, B_C, B_D, s_1, 8)

In [11]:
print((np.array(imgb_d)==np.array(imgb_d_hk)).all())

True


In [12]:
arr2_hk = From_blocks_to_arr_image(imgb_d_hk,Nb,k)

im_1 = Image.fromarray(arr2_hk, 'L')
im_1.save("decrypted_hk.jpg")